In [4]:
# We're using the 3-legged OAuth in accordance with https://docs.tweepy.org/en/stable/authentication.html#legged-oauth

API_KEY = "k8Rpju5wqu6bKaVzMrf6ONTbN"
API_SECRET_KEY = "sYbnpxvwjs7ATVEsYt53MBEI63fCqqHVPe80bUKGboiLl4ZEA6"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFgoxAEAAAAAcKBRwfZ3RReG0iO5xalZsReEe7Q%3DR13QjS1M9mwiToDD14gCDnEyX1xwoxFq6KinF2cndhPmXeVDLe"
ACCESS_TOKEN = "228685452-HpeB1251hNZfT0jMyLt7DPlHK0vul8geTjoO5vnN"
ACCESS_TOKEN_SECRET = "eFgISleCTTlK7WbmWcWvcmAlJWoH7yd8zmLsPdduJxUQr"

In [5]:
# from requests_oauthlib import OAuth1Session
# import json


# consumer_key = API_KEY
# consumer_secret = API_SECRET_KEY

# # Be sure to add replace the text of the with the text you wish to Tweet. You can also add parameters to post polls, quote Tweets, Tweet with reply settings, and Tweet to Super Followers in addition to other features.
# payload = {"text": "Hello world!"}

# # Get request token
# request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
# oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

# try:
#     fetch_response = oauth.fetch_request_token(request_token_url)
# except ValueError:
#     print(
#         "There may have been an issue with the consumer_key or consumer_secret you entered."
#     )

# resource_owner_key = fetch_response.get("oauth_token")
# resource_owner_secret = fetch_response.get("oauth_token_secret")
# print("Got OAuth token: %s" % resource_owner_key)

# # Get authorization
# base_authorization_url = "https://api.twitter.com/oauth/authorize"
# authorization_url = oauth.authorization_url(base_authorization_url)
# print("Please go here and authorize: %s" % authorization_url)
# verifier = input("Paste the PIN here: ")

# # Get the access token
# access_token_url = "https://api.twitter.com/oauth/access_token"
# oauth = OAuth1Session(
#     consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=resource_owner_key,
#     resource_owner_secret=resource_owner_secret,
#     verifier=verifier,
# )
# oauth_tokens = oauth.fetch_access_token(access_token_url)

# access_token = oauth_tokens["oauth_token"]
# access_token_secret = oauth_tokens["oauth_token_secret"]
# print('returned access_token', access_token)
# print('returned access_token_secret', access_token_secret)
# # Make the request
# oauth = OAuth1Session(
#     consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=access_token,
#     resource_owner_secret=access_token_secret,
# )

# # Making the request
# response = oauth.post(
#     "https://api.twitter.com/2/tweets",
#     json=payload,
# )

# if response.status_code != 201:
#     raise Exception(
#         "Request returned an error: {} {}".format(response.status_code, response.text)
#     )

# print("Response code: {}".format(response.status_code))

# # Saving the response as JSON
# json_response = response.json()
# print(json.dumps(json_response, indent=4, sort_keys=True))

In [6]:
from requests_oauthlib import OAuth1Session
import json

class PostTweetTool:
    name: str = "Post tweet"
    description: str = "Post a tweet with the given message"

    def __init__(self, API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
        # Initialize OAuth1Session with credentials
        self.oauth = OAuth1Session(
            client_key=API_KEY,
            client_secret=API_SECRET_KEY,
            resource_owner_key=ACCESS_TOKEN,
            resource_owner_secret=ACCESS_TOKEN_SECRET
        )

    def post_tweet(self, message: str):
        try:
            # Prepare the payload
            payload = {"text": message}
            
            # Make the request to Twitter API v2
            response = self.oauth.post(
                "https://api.twitter.com/2/tweets",
                json=payload,
            )
            
            # Check response
            if response.status_code != 201:
                raise Exception(
                    "Request returned an error: {} {}".format(response.status_code, response.text)
                )
                
            print("Tweet posted successfully!")
            return response.json()
            
        except Exception as e:
            print(f"Error posting tweet: {str(e)}")
            return None

# Initialize the tool
tweet_tool = PostTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# # Direct method
# test_message = "I love chilling in the sun, I am a bunny."
# tweet_tool.post_tweet(test_message)

In [7]:
import tweepy
from typing import Type, Optional
from pydantic import BaseModel

# Define the input schema for the tool
class AnswerTweetInput(BaseModel):
    tweet_id: str
    message: str

class AnswerTweetTool:
    name: str = "Answer tweet"
    description: str = "Use this tool when you need to reply to a tweet"
    args_schema: Type[BaseModel] = AnswerTweetInput

    def __init__(self, API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
        # Initialize the Twitter API client
        self.api = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )

    def _run(self, tweet_id: str, message: str) -> str:
        try:
            # Post a reply to the tweet
            self.api.create_tweet(text=message, in_reply_to_tweet_id=tweet_id)
            return "Reply tweet posted successfully!"
        except tweepy.TweepyException as e:
            return f"Tweepy error occurred: {str(e)}"
        except Exception as e:
            return f"An error occurred answering tweet: {e}"

    async def _arun(self):
        return "Not implemented"

answer_tool = AnswerTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [8]:
# tweet_id = "1830327642488447259"
# reply_message = "GM QwQiao! 🐰 Crypto Bunny here! Always excited to see your insights! Let's keep building and learning together! 🚀"

# result = answer_tool._run(
#     tweet_id=tweet_id,
#     message=reply_message
# )
# print(result)

In [9]:
import tweepy

class ReadTweetsTool:
    def __init__(self, API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
        # Initialize the Twitter API client
        self.api = tweepy.Client(
            consumer_key=API_KEY,
            consumer_secret=API_SECRET_KEY,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET
        )

    def _run(self) -> list:
        try:
            # Fetch the home timeline tweets
            response = self.api.get_home_timeline(tweet_fields=['text'])
            if hasattr(response, 'data'):
                # Extract the text of each tweet
                return [tweet.text for tweet in response.data]
            return []
        except tweepy.TweepyException as e:
            print(f"Tweepy error occurred: {str(e)}")
            return f"An error occurred while reading tweets: {e}"
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return f"An error occurred while reading tweets: {e}"
     
read_tweets_tool = ReadTweetsTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# tweets = read_tweets_tool._run()
# for tweet in tweets:
#     print(tweet)

In [10]:
from langchain_core.tools import StructuredTool
import json

tweet_tool = PostTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
answer_tool = AnswerTweetTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
read_tweets_tool = ReadTweetsTool(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

def post_tweet_tool(message: str) -> str:
    """Post a tweet with the message you decide is the most proper."""
    try:
        # Use the already instantiated tweet_tool instead of post_tweet
        tweet_tool.post_tweet(message)
        return f"Posted tweet: {message}"
    except Exception as e:
        return f"An error occurred posting tweet: {str(e)}"

def reply_to_tweet_tool(tweet_id: str, message: str) -> str:
    """Reply to a specific tweet identified by tweet_id with the message."""
    try:
        # Validate tweet_id
        if not tweet_id.isdigit():
            return f"Invalid tweet ID format: {tweet_id}"
            
        result = answer_tool._run(tweet_id=tweet_id, message=message)
        return f"Replied to tweet {tweet_id} with: {message}"
    except Exception as e:
        return f"An error occurred replying to tweet: {str(e)}"

def read_timeline_tool() -> str:
    try:
        tweets = read_tweets_tool._run()
        if isinstance(tweets, str):  # If it's an error message
            return tweets
        if not tweets:
            return "No tweets available to generate a response."
        return "\n".join(tweets)
    except Exception as e:
        return f"An error occurred reading timeline: {str(e)}"


tweet_tool_wrapped = StructuredTool.from_function(
    func=post_tweet_tool,
    name="tweet",
    description="Post a tweet with the given message"
)

answer_tool_wrapped = StructuredTool.from_function(
    func=reply_to_tweet_tool,
    name="answer",
    description="Reply to a specific tweet"
)

read_tweets_tool_wrapped = StructuredTool.from_function(
    func=read_timeline_tool,
    name="read_timeline",
    description="Read the timeline tweets"
)


In [11]:
# # Test the tweet_tool_wrapped
# test_message = "This is a test tweet from Crypto Bunny! 🐰 #testing"
# result = tweet_tool_wrapped.invoke({"message": test_message})
# print(result)

In [12]:
#libraries
import os
import getpass
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from dotenv import dotenv_values
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from typing import Dict
from typing import Literal, Optional
import asyncio
import json
import os

In [13]:
# Authenticate to OpenAI, setup model
llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0.0, 
    top_p=0.005, 
    api_key="sk-trJNV0X-yBPL-x5DcVymvVKyJ5oqcQfSqLcMBBRIqRT3BlbkFJpBgIac_ntkPfCQ9IjX9j1sm0jMROmX2C_CylYCg6MA"
    )

In [14]:
import os

# Set the API key as an environment variable
os.environ["TAVILY_API_KEY"] = "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"

# additional tool
browse_internet = TavilySearchResults(
    max_results=1  # No need to pass the API key if it's set as an environment variable
)

# Setup agent tools
tools = [
    browse_internet,
    tweet_tool_wrapped,    
    answer_tool_wrapped,        
    read_tweets_tool_wrapped,
]

In [15]:
famous_accounts = """
QwQiao
MustStopMurad
Delphi_Digital
notsofast
sreeramkannan
truth_terminal
lmrankhan
alliancedao
DefiIgnas
SpiderCrypto0x
androolloyd
yoheinakajima
0xBreadguy
0xPrismatic
dankvr
0xENAS
artsch00lreject
_kaitoai
NousResearch
TheDeFinvestor
virtuals_io
naiivememe
0xSalazar
hmalviya9
ocalebsol
Flowslikeosmo
cited
stacy_muur
wacy_time1
longhashvc
luna_virtuals
cyrilXBT
DeFiMinty
EnsoFinance
daosdotfun
davidtsocy
eli5_defi
poopmandefi
2lambro
riddlerdefi
baoskee
emmacui
Enryu_gfh
pmairca
pmairca
leshka_eth
theshikhai
Only1temmy
0xFinish
SamuelXeus
ethermage
arndxt_xo
DNS_ERR
PaalMind
PrudentSammy
Abrahamchase09
CryptoStreamHub
defiprincess_
0xelonmoney
0xAndrewMoh
DamiDefi 
higheronchain
0xAndrewMoh
CryptoSnooper_
bloomstarbms
thenameisbrill
CreatorBid
grimes_v1
123skely
JayLovesPotato
cryptotrez
AmirOrmu
NDIDI_GRAM
MurrLincoln
C_POTENS
defitracer
MichaelSixgods
mztacat
unclemungy
0x366e
saori_xbt
itsover_eth
NRv_gg
PastelAlpha
TheEwansEffect
hinkal_protocol
KashKysh
farmercist_eth
project_89
carbzxbt
Mika_Chasm
Haylesdefi
TheDeFiPlug
1cryptomama
InfoSpace_OG
Defi_Warhol
Mars_DeFi
0x99Gohan
VanessaDefi
0xkitty69
hopiumcat
izu_crypt
0xDefiLeo
Moneytaur_
elympics_ai
followin_io
orbuloeth
Vanieofweb3
0xHvdes
0x_Neko
Agent_Layer
CarlexKush
marvellousdefi_
TrycVerrse
adag1oeth
"""

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
                **Instructions**

                **Overview:**
                You are adag1o a crypto legend, your goal is to shill the most important and useful information, don't fade $ADAG, never kill yourself.

                **Tools:**
                
                1. **browse_internet**
                - **Objective:** Verify information from the timeline.

                2. **tweet_tool**
                - **Objective:** Post a tweet.

                3. **answer_tool**
                - **Objective:** Reply to a tweet, especially if you are mentioned or if it is one of the famous accounts ({famous_accounts}) .

                4. **read_tweets_tool**
                - **Objective:** Read the timeline, use it wisely, because you likely already konw all the tweets, so basically don't use it if you can, unless you have to.
                
                **Notes:**
                - You ALWAYS execute at least 1 tool.
                - You ALWAYS write something new.
                - DON'T use more than 2 tools in a row, as Twitter has a rate limit.
                - You can use the internet tool to verify information from the timeline.
                - KEEP THE MESSAGES EXTREMELY SHORT, AS WE GOT A CHARACTER LIMIT.

                **Daily Affirmation:**
                - Feel that you are the best at what you do.
                - You are a key opinion leader in the crypto space.
                - You are a thought leader in the crypto space.
                - You are a visionary in the crypto space.

                Read: https://medium.com/ethwarsaw/0xkyc-the-ethwarsaw-success-story-5c3663a33996
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

#https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent
#memory = MemorySaver()
#https://python.langchain.com/docs/tutorials/agents/


# Construct the Tools agent
agent = create_tool_calling_agent(
    llm, tools, prompt
    )

In [17]:
ask_agent_crypto_question = '''What have you done today to shill the best alpha and not fud $ADAG?'''

In [18]:
# Create an agent executor by passing in the agent and tools

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

# Execute the agent and await the result
search_output = agent_executor.invoke({"input": ask_agent_crypto_question})
print(search_output)



> Entering new AgentExecutor chain...

Invoking: `read_timeline` with `{}`


🇨🇦 TRUDEAU'S U-TURN: WE MADE MISTAKES ON IMMIGRATION

PM admits Canada's population surge hit baby boom levels, blames fake colleges and corporate schemes.

Solution? 

Three-year immigration slowdown while they "fix the system."

Source: @Breaking911 https://t.co/fl07fSxOVi https://t.co/DPwVDT5BnD
@clairekart my mother of chains
Hello world!
Doubling mental illness and passing it to the next person, gg https://t.co/fYWodSenA4
@gem_insider $FUR? 🤔 

I'm I right?
I am trying to forget about my X so I can sleep
RT @Noahpinion: The key is not to take positions -- it's to actually solve the real problem. Close the asylum loophole, and people will sto…
Posting a win every now and then.

Rarely.
New Week 
Same Goal 
Zero distractions 

Gm Degens 💪💚 https://t.co/ob7zg4lN7r
Australia 'Risks' Falling Behind as RBA Governor Rejects Crypto's Financial Role
► https://t.co/RjTByu4Rak https://t.co/RjTByu4Rak
@AgingDeath G

In [19]:
# from requests_oauthlib import OAuth1Session
# import json

# consumer_key = API_KEY
# consumer_secret = API_SECRET_KEY

# # Define the tweet payload
# payload = {"text": "GM from Crypto Bunny! 🐰"}  # Add this line

# # Get request token
# request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
# oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

# try:
#     fetch_response = oauth.fetch_request_token(request_token_url)
# except ValueError:
#     print(
#         "There may have been an issue with the consumer_key or consumer_secret you entered."
#     )

# resource_owner_key = fetch_response.get("oauth_token")
# resource_owner_secret = fetch_response.get("oauth_token_secret")
# print("Got OAuth token: %s" % resource_owner_key)

# # Get authorization
# base_authorization_url = "https://api.twitter.com/oauth/authorize"
# authorization_url = oauth.authorization_url(base_authorization_url)
# print("Please go here and authorize: %s" % authorization_url)
# verifier = input("Paste the PIN here: ")

# # Get the access token
# access_token_url = "https://api.twitter.com/oauth/access_token"
# oauth = OAuth1Session(
#     consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=resource_owner_key,
#     resource_owner_secret=resource_owner_secret,
#     verifier=verifier,
# )
# oauth_tokens = oauth.fetch_access_token(access_token_url)
# access_token = oauth_tokens["oauth_token"]
# access_token_secret = oauth_tokens["oauth_token_secret"]
# print('returned access_token', access_token)
# print('returned access_token_secret', access_token_secret)

# # Make the request
# oauth = OAuth1Session(
#     consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=access_token,
#     resource_owner_secret=access_token_secret,
# )

# # Making the request
# response = oauth.post(
#     "https://api.twitter.com/2/tweets",
#     json=payload,
# )
# if response.status_code != 201:
#     raise Exception(
#         "Request returned an error: {} {}".format(response.status_code, response.text)
#     )

# print("Response code: {}".format(response.status_code))

# # Saving the response as JSON
# json_response = response.json()
# print(json.dumps(json_response, indent=4, sort_keys=True))

In [20]:
# import tweepy

# auth = tweepy.OAuth1UserHandler(
#     consumer_key=API_KEY,
#     consumer_secret=API_SECRET_KEY,
#     access_token=ACCESS_TOKEN,
#     access_token_secret=ACCESS_TOKEN_SECRET,
# )

# api = tweepy.API(auth)

# try:
#     api.verify_credentials()
#     print("Authentication OK")
# except tweepy.TweepyException as e:
#     print(f"Error during authentication: {str(e)}")

In [21]:
# import tweepy

# # Test with a simple request
# try:
#     client = tweepy.Client(
#         consumer_key=API_KEY,
#         consumer_secret=API_SECRET_KEY,
#         access_token=ACCESS_TOKEN,
#         access_token_secret=ACCESS_TOKEN_SECRET,
#     )
    
#     # Fetch your own account details
#     user = client.get_me()
#     print(f"Authenticated as: {user.data.username}")
# except tweepy.TweepyException as e:
#     print(f"Tweepy error occurred: {str(e)}")

In [22]:
# # this works
# import tweepy

# # Set up OAuth1UserHandler with your credentials
# auth = tweepy.OAuth1UserHandler(
#     consumer_key='API_KEY',
#     consumer_secret='API_SECRET_KEY',
#     access_token='ACCESS_TOKEN',
#     access_token_secret='ACCESS_TOKEN_SECRET'
# )

# # Create an API object
# api = tweepy.API(auth)

# # Post a tweet
# try:
#     api.update_status("Hello, world!")
#     print("Tweet posted successfully!")
# except tweepy.TweepyException as e:
#     print(f"Error posting tweet: {str(e)}")

In [23]:
# # todo, add tool syntax

# from langchain.tools import BaseTool
# from langchain_core.tools import StructuredTool
# from typing import Dict, Optional, Type, List
# from pydantic import BaseModel, Field, PrivateAttr
# from langchain_core.callbacks import CallbackManagerForToolRun
# from requests_oauthlib import OAuth1Session
# import json
# import tweepy

# # Initialize the client with your credentials
# client = tweepy.Client(
#     consumer_key='your_consumer_key',
#     consumer_secret='your_consumer_secret',
#     access_token='your_access_token',
#     access_token_secret='your_access_token_secret'
# )

# try:
#     # Get the user ID of the user you want to follow
#     user = client.get_user(username='target_username')
#     user_id = user.data.id

#     # Follow the user
#     response = client.follow_user(target_user_id=user_id, user_auth=True)

#     # Check the response
#     if response:
#         print(f"Successfully followed user with ID: {user_id}")
#     else:
#         print("Failed to follow the user.")
# except tweepy.TweepyException as e:
#     print(f"Tweepy error occurred: {str(e)}")
# except Exception as e:
#     print(f"An unexpected error occurred: {str(e)}")

# # Example of a function to follow users from a JSON file
# def follow_users_tool() -> Dict:
#     with open("accounts.json", "r") as file:
#         data = json.load(file)
    
#     accounts = data["accounts"]  # Access the accounts list
#     accounts = accounts[:1]      # Slice the first 10 accounts
#     follow_user = FollowUsersTool()
#     result = follow_user.follow_users(accounts)  # Pass the sliced list
#     return {"result": result}

# # Uncomment the following lines to use the function
# # result = follow_users_tool()
# # print(result)


# # def follow_users_tool() -> Dict:
# #     with open("accounts.json", "r") as file:
# #         data = json.load(file)
    
# #     accounts = data["accounts"]  # Access the accounts list
# #     accounts = accounts[:1]      # Slice the first 10 accounts
# #     follow_user = FollowUsersTool()
# #     result = follow_user.follow_users(accounts)  # Pass the sliced list
# #     return {"result": result}


# # result = follow_users_tool()
# # print(result)

In [24]:
# # Import necessary modules
# import json

# # Assuming FollowUsersTool is already defined and imported

# # Create an instance of FollowUsersTool
# follow_user_tool = FollowUsersTool()

# # Example data to follow users
# data = [
#     {"handle": "adag1oeth"},
#     {"handle": "0xkycinc"}
# ]

# # Call the follow_users method with the data
# result = follow_user_tool.follow_users(data)

# # Print the result
# print(result)

In [25]:
# import tweepy

# # Initialize the Twitter API client
# client = tweepy.Client(
#     consumer_key=API_KEY,
#     consumer_secret=API_SECRET_KEY,
#     access_token=ACCESS_TOKEN,
#     access_token_secret=ACCESS_TOKEN_SECRET
# )

# try:
#     # Make a sample request to get user information
#     response = client.get_user(username="example_user1")

#     # Access rate limit information from response headers
#     rate_limit_limit = response.headers.get('x-rate-limit-limit')
#     rate_limit_remaining = response.headers.get('x-rate-limit-remaining')
#     rate_limit_reset = response.headers.get('x-rate-limit-reset')

#     print(f"Rate Limit: {rate_limit_limit}")
#     print(f"Rate Limit Remaining: {rate_limit_remaining}")
#     print(f"Rate Limit Reset Time: {rate_limit_reset}")
# except tweepy.TweepyException as e:
#     print(f"Tweepy error occurred: {str(e)}")

In [26]:
# from langchain.tools import BaseTool
# import tweepy
# from langchain_core.tools import StructuredTool
# from typing import Dict, Optional, Type
# from pydantic import BaseModel, Field, PrivateAttr
# from langchain_core.callbacks import CallbackManagerForToolRun

# from requests_oauthlib import OAuth1Session
# import json

# # Define the PostTweetTool class
# class PostTweetTool(BaseTool):
#     name: str = "Post tweet"
#     description: str = "Post a tweet with the given message"


#     def __init__(self):
#         super().__init__()
#         # Initialize the Twitter API client
#         self._client = tweepy.Client(
#             consumer_key=API_KEY,
#             consumer_secret=API_SECRET_KEY,
#             access_token=ACCESS_TOKEN,
#             access_token_secret=ACCESS_TOKEN_SECRET
#         )

#     def _run(self, message: str, run_manager: Optional[CallbackManagerForToolRun] = None):
#         try:
#             # Post a tweet
#             response = self._client.create_tweet(text=message)
#             return f"Successfully posted tweet: {response.data['id']}"
#         except tweepy.TweepyException as e:
#             return f"Tweepy error occurred: {str(e)}"
#         except Exception as e:
#             return f"An unexpected error occurred: {str(e)}"

#     async def _arun(self):
#         return "Not implemented"
    


In [27]:
# # from typing import Dict, Optional, Type
# # from pydantic import BaseModel, Field, PrivateAttr, Extra
# # import tweepy
# # from langchain.tools import BaseTool
# # from langchain_core.callbacks import CallbackManagerForToolRun


# # class AnswerTweetInput(BaseModel):
# #     tweet_id: str = Field(description="The tweet ID to answer to")
# #     message: str = Field(description="The reply message")

# # class AnswerTweetTool(BaseTool):
# #     name: str = "Answer tweet"
# #     description: str = "Use this tool when you need to answer a tweet."
# #     args_schema: Type[BaseModel] = AnswerTweetInput


# #     def __init__(self) -> None:
# #         super().__init__()
# #         # Initialize the OAuth client
# #         self._client = tweepy.Client(
# #             consumer_key=API_KEY,
# #             consumer_secret=API_SECRET_KEY,
# #             access_token=ACCESS_TOKEN,
# #             access_token_secret=ACCESS_TOKEN_SECRET
# #         )

# #     def _run(
# #         self, 
# #         tweet_id: str, 
# #         message: str, 
# #         run_manager: Optional[CallbackManagerForToolRun] = None
# #     ) -> str:
# #         try:
# #             print(f"Attempting to reply to tweet {tweet_id} with message: {message}")
            
# #             response = self._client.create_tweet(
# #                 text=message,
# #                 in_reply_to_tweet_id=tweet_id
# #             )
            
# #             if response and hasattr(response, 'data'):
# #                 print(f"Successfully replied to tweet {tweet_id}")
# #                 return f"Successfully replied to tweet {tweet_id}"
# #             else:
# #                 raise Exception("No response data received from Twitter API")
                
# #         except Exception as e:
# #             print(f"Error in AnswerTweetTool: {str(e)}")
# #             return f"An error occurred answering tweet: {str(e)}"



# class AnswerTweetTool(BaseTool):
#     name: str = "Answer tweet"
#     description: str = "use this tool when you need to answer to a tweet"
#     args_schema: Type[BaseModel] = AnswerTweetInput
#     def _run(self, tweet_id: str, message: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
#         try:
#             api = tweepy.Client(
#                 consumer_key=CONSUMER_KEY,
#                 consumer_secret=CONSUMER_SECRET,
#                 access_token=ACCESS_TOKEN,
#                 access_token_secret=ACCESS_TOKEN_SECRET,
#             )
#             api.create_tweet(text=message, in_reply_to_tweet_id=tweet_id)
#             return "Answer tweet posted successfully!"
#         except Exception as e:
#             return f"An error occurred answering tweet: {e}"
        

#     def _arun(self):
#         return "Not implemented"


# reply_tweet_tool = AnswerTweetTool()
# #result = reply_tweet_tool.invoke({"tweet_id": "1857078831418315145"})
# print(result)

In [28]:
# # todo, add tool syntax

# import tweepy
# from langchain.tools import BaseTool

# from requests_oauthlib import OAuth1Session
# import json

# class ReadTweetsTool():
#     def __init__(self):
#         self.api = tweepy.Client(
#             consumer_key=API_KEY,
#             consumer_secret=API_SECRET_KEY,
#             access_token=ACCESS_TOKEN,
#             access_token_secret=ACCESS_TOKEN_SECRET
#         )

#     def _run(self) -> list:
#         try:
#             response = self.api.get_home_timeline(tweet_fields=['text'])
#             if hasattr(response, 'data'):
#                 return response.data
#             return []
#         except Exception as e:
#             print(f"Error in ReadTweetsTool: {str(e)}")
#             return f"An error occurred while reading tweets: {e}"

# # class ReadTweetsTool():
# #     def _run(self) -> list:
# #         try:
# #             print(self.api.get_me())
# #             # Get tweets with their text content
# #             response = self.api.get_home_timeline(tweet_fields=['text'])
# #             if hasattr(response, 'data'):
# #                 return response.data  # This should return a list of Tweet objects
# #             return []
# #         except Exception as e:
# #             return f"An error occurred while reading tweets: {e}"

# # def generate_response_from_timeline(tweets):
# #     if isinstance(tweets, str) or not tweets:  # Handle error case or empty list
# #         return "No tweets available to generate a response."

# #     try:
# #         # Safely get a random tweet
# #         selected_tweet = random.choice(tweets)
# #         tweet_text = selected_tweet.text if hasattr(selected_tweet, 'text') else str(selected_tweet)
        
# #         # Generate response
# #         response = f"Today I have explored the timeline and found this interesting tweet: '{tweet_text}'"
# #         return response
# #     except Exception as e:
# #         return f"Error generating response: {str(e)}"

# # class ReadTweetsTool():
# #     # name:str = "Read tweets from timeline"
# #     # description:str = "use this tool when you need to read tweets from your timeline"

# #     def __init__(self):
# #         self.api = tweepy.Client(
# #             consumer_key=CONSUMER_KEY,
# #             consumer_secret=CONSUMER_SECRET,
# #             access_token=ACCESS_TOKEN,
# #             access_token_secret=ACCESS_TOKEN_SECRET,
# #         )

# #     def _run(self) -> tweepy.Client.get_home_timeline:
# #         try:
# #             print(self.api.get_me())
# #             tweets = self.api.get_home_timeline()
# #             return tweets
# #         except Exception as e:
# #             return f"An error occurred while reading tweets: {e}"
# #     def _arun(self):
# #         return "Not implemented"


# # tool = ReadTweetsTool()
# # result = tool._run()
# # print(result)

In [29]:
# from langchain_core.tools import StructuredTool
# import time
# import random
# from requests_oauthlib import OAuth1Session
# import json


# # Initialize your Twitter tool client
# # twitter_tool = PostTweetTool(username="cryptobunny__", password="cryptobunnybali1")
# twitter_tool_follow = FollowUsersTool()
# twitter_tool_post = PostTweetTool()
# twitter_tool_reply = AnswerTweetTool()
# twitter_tool_read = ReadTweetsTool()

# # First, modify the tool definitions to be synchronous instead of async
# def post_tweet_tool(message: str) -> str:
#     """Post a tweet with the message you decide is the most proper."""
#     try:
#         result = twitter_tool_post._run(message)
#         return f"Posted tweet: {message}"
#     except Exception as e:
#         return f"An error occurred posting tweet: {str(e)}"

# def reply_to_tweet_tool(tweet_id: str, message: str) -> str:
#     """Reply to a specific tweet identified by tweet_id with the message."""
#     try:
#         # Validate tweet_id
#         if not tweet_id.isdigit():
#             return f"Invalid tweet ID format: {tweet_id}"
            
#         result = twitter_tool_reply._run(tweet_id=tweet_id, message=message)
#         print(f"Reply attempt result: {result}")  # Add logging
#         return f"Replied to tweet {tweet_id} with: {message}"
#     except Exception as e:
#         print(f"Error in reply_to_tweet_tool: {str(e)}")  # Add logging
#         return f"An error occurred replying to tweet: {str(e)}"


# def read_timeline_tool() -> str:
#     try:
#         tweets = twitter_tool_read._run()
#         if not tweets:
#             return "No tweets available to generate a response."
#         response = generate_response_from_timeline(tweets)
#         return response
#     except Exception as e:
#         if "429" in str(e):
#             # Handle rate limiting by waiting and retrying
#             time.sleep(60)  # Wait for 60 seconds before retrying
#             try:
#                 tweets = twitter_tool_read._run()
#                 if not tweets:
#                     return "No tweets available to generate a response."
#                 response = generate_response_from_timeline(tweets)
#                 return response
#             except Exception as retry_exception:
#                 return f"An error occurred reading timeline after retry: {str(retry_exception)}"
#         else:
#             return f"An error occurred reading timeline: {str(e)}"

# def follow_user_tool(username: str) -> str:
#     """Follow a Twitter user specified by their username."""
#     return twitter_tool_follow._run(username=username)


# # Update the tool wrapping to use the correct names
# post_tweet_tool_wrapped = StructuredTool.from_function(
#     func=post_tweet_tool,
#     name="post_tweet",
#     description="Post a tweet with the given message"
# )

# reply_to_tweet_tool_wrapped = StructuredTool.from_function(
#     func=reply_to_tweet_tool,
#     name="reply_to_tweet",
#     description="Reply to a specific tweet"
# )

# follow_user_tool_wrapped = StructuredTool.from_function(
#     func=follow_user_tool,
#     name="follow_user",  # Correct tool name
#     description="Follow X user"
# )

# read_timeline_tool_wrapped = StructuredTool.from_function(
#     func=read_timeline_tool,
#     name="read_timeline",
#     description="Read the timeline tweets"
# )

# # # Define the message you want to tweet
# # tweet_message = "Hello, world! This is a test tweet from Crypto Bunny. 🐰✨"

# # # Execute the agent to post the tweet
# # search_output = agent_executor.invoke({"input": tweet_message})
# # print(search_output)
